In [3]:
import numpy as np
import pandas as pd
import scipy as scp
import itertools as it

In [ ]:
class sys_resolve_atoms(self, n):

    def __init__(self, n):
        self.matrix = np.zeros((n, n), dtype=float)
        self.n = n

    def add_matrix(self, matrix):
        if matrix.shape[0] != self.matrix.shape[0] or matrix.shape[1] != self.matrix.shape[1]:
            raise ValueError("Matrix dimensions do not match.")
        self.matrix += matrix

    def create_matrix(self, m: list, k: list):

        dict_mk = {}

        k = {}
        
        for i in range(self.n):
            k[f"{i+1}{i+2}"] = k[i]
            
        for i in it.product(range(self.n), repeat=2):
            if (i[0] == 0 and i[1] == 0) or (i[0] == self.n-1 and i[1] == self.n-1):
                self.matrix[i[0], i[1]] = (k[i]/m[i])
            elif i == j:
                self.matrix[i, j] = (k[i]+k[i+1])/m[i]

            else:
                self.matrix[i, j] = k[i] * k[j]

    

In [14]:
class sys_resolve_atoms:
    def __init__(self, n):
        """
        Inicializa o sistema com n massas.

        Args:
            n (int): O número de massas (e o tamanho da matriz).
        """
        if n < 2:
            raise ValueError("O número de massas (n) deve ser pelo menos 2.")
        self.n = n
        self.matrix = np.zeros((n, n), dtype=float)
        self._autovalores = None
        self._autovetores = None

    def create_matrix(self, m: list, k: list):
        """
        Gera a matriz dinâmica D com base nas massas (m) e nas constantes de mola (k).
        """

        if len(m) != self.n:
            raise ValueError(f"A lista de massas 'm' deve ter {self.n} elementos, mas tem {len(m)}.")
        if len(k) != self.n - 1:
            raise ValueError(f"A lista de constantes 'k' deve ter {self.n - 1} elementos, mas tem {len(k)}.")

        
        self.matrix = np.zeros((self.n, self.n), dtype=float)

        for i in range(self.n):
            for j in range(self.n):
                if i == j:
                    if i == 0:
                        self.matrix[i, j] = k[0] / m[i]
                    elif i == self.n - 1:
                        self.matrix[i, j] = k[i - 1] / m[i]
                    else:
                        self.matrix[i, j] = (k[i - 1] + k[i]) / m[i]
                elif abs(i - j) == 1:
                    k_index = min(i, j)
                    self.matrix[i, j] = -k[k_index] / m[i]
        
        return self.matrix

    def _calculate_system_values(self):
        """
        Método interno para calcular autovalores e autovetores
        """
        autovalores, autovetores = np.linalg.eig(self.matrix)
        
        sorted_indices = np.argsort(autovalores)
        self._autovalores = autovalores[sorted_indices]
        self._autovetores = autovetores[:, sorted_indices]

    def get_autovalores(self):
        """
        Calcula e retorna os autovalores (λ = ω²) da matriz dinâmica D.
        """
        self._calculate_system_values()
        return self._autovalores

    def get_autovetores(self):
        """
        Calcula e retorna os autovetores (modos normais de vibração) da matriz D.

        Returns:
            np.ndarray: Uma matriz onde cada coluna é um autovetor correspondente
                        a um autovalor (na mesma ordem retornada por get_autovalores).
        """
        self._calculate_system_values()
        return self._autovetores


In [ ]:
# --- Exemplo de Uso ---

# Para n = 2 massas (Matriz 2x2)
print("--- Matriz para n=2 ---")
n2 = 2
m2 = [1.0, 2.0]  # m1, m2
k2 = [10.0]      # k12
sys2 = sys_resolve_atoms(n2)
matrix2 = sys2.create_matrix(m=m2, k=k2)
print(matrix2)
# Saída esperada:
# [[ 10.  -10. ]
#  [ -5.   5. ]]

# Para n = 3 massas (Matriz 3x3)
print("\n--- Matriz para n=3 ---")
n3 = 3
m3 = [1.0, 2.0, 1.5] # m1, m2, m3
k3 = [10.0, 20.0]    # k12, k23
sys3 = sys_resolve_atoms(n3)
matrix3 = sys3.create_matrix(m=m3, k=k3)
print(matrix3)
# Saída esperada:
# [[ 10.         -10.          0.        ]
#  [ -5.          15.         -10.        ]
#  [  0.         -13.333...   13.333...]]

# Para n = 4 massas (Matriz 4x4)
print("\n--- Matriz para n=4 ---")
n4 = 4
m4 = [1.0, 2.0, 1.5, 3.0] # m1, m2, m3, m4
k4 = [10.0, 20.0, 5.0]    # k12, k23, k34
sys4 = sys_resolve_atoms(n4)
matrix4 = sys4.create_matrix(m=m4, k=k4)
print(matrix4)
# Saída esperada:
# [[ 10.          -10.           0.           0.        ]
#  [ -5.           15.          -10.           0.        ]
#  [  0.          -13.333...    16.666...     -3.333... ]
#  [  0.           0.           -1.666...      1.666...  ]]

In [15]:

# Sistema de 3 massas e 2 molas (como na imagem)
n = 3
# Valores hipotéticos para massas e constantes de mola
masses = [1.0, 1.0, 1.0]  # m1, m2, m3 (em kg)
spring_constants = [1.0, 1.0] # k12, k23 (em N/m)

# 1. Criar a instância do sistema
system = sys_resolve_atoms(n)

# 2. Gerar a matriz dinâmica D
D_matrix = system.create_matrix(m=masses, k=spring_constants)
print("--- Matriz Dinâmica D ---")
print(D_matrix)

# 3. Calcular e imprimir os autovalores (ω²)
# Os autovalores correspondem ao quadrado das frequências angulares de oscilação
autovalores = system.get_autovalores()
print("\n--- Autovalores (λ = ω²) ---")
print(autovalores)

# 4. Calcular e imprimir os autovetores (Modos Normais)
# Cada coluna da matriz de autovetores representa um modo normal de vibração
autovetores = system.get_autovetores()
print("\n--- Autovetores (Modos Normais de Vibração) ---")
print(autovetores)

# 5. Calcular as frequências angulares (ω)
# ω = sqrt(λ), mas apenas para autovalores positivos
angular_frequencies = np.sqrt(autovetores[autovetores > 0])
print("\n--- Frequências Angulares de Oscilação (ω) ---")
print(angular_frequencies)

--- Matriz Dinâmica D ---
[[ 1. -1.  0.]
 [-1.  2. -1.]
 [ 0. -1.  1.]]

--- Autovalores (λ = ω²) ---
[-5.13860489e-17  1.00000000e+00  3.00000000e+00]

--- Autovetores (Modos Normais de Vibração) ---
[[ 5.77350269e-01 -7.07106781e-01 -4.08248290e-01]
 [ 5.77350269e-01  2.48915666e-16  8.16496581e-01]
 [ 5.77350269e-01  7.07106781e-01 -4.08248290e-01]]

--- Frequências Angulares de Oscilação (ω) ---
[7.59835686e-01 7.59835686e-01 1.57770614e-08 9.03602004e-01
 7.59835686e-01 8.40896415e-01]
